In [ ]:
%pip install ultralytics
%pip install supervision

In [ ]:
import numpy as np
import os
from ultralytics import YOLO
import supervision as sv

# Only on Google colab

In [ ]:
!git clone https://github.com/DanielLankry/Football-Vision-Stats-ML

In [ ]:
os.chdir('/content/Football-Vision-Stats-ML')
!pwd
os.chdir('Football Advanced stats with python')
!pwd
!ls

In [ ]:
PLAYER_DETECTION_MODEL = YOLO("yolo11m.pt")
SOURCE_VIDEO_PATH = "121364_0.mp4"

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

sv.plot_image(frame)

In [ ]:

tracker = sv.ByteTrack()

video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

bounding_box = sv.BoxAnnotator()
label = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()



def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = PLAYER_DETECTION_MODEL(frame)[0]
    detections = sv.Detections.from_ultralytics(results)

    tracked_pos = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {class_name}"
        for class_name, tracker_id
        in zip(detections.data["class_name"], detections.tracker_id)
    ]

    annotated_frame = bounding_box.annotate(
        frame.copy(), detections=detections)
    annotated_frame = label.annotate(
        annotated_frame, detections=detections, labels=labels)
    return trace_annotator.annotate(
        annotated_frame, detections=detections)

sv.process_video(source_path=SOURCE_VIDEO_PATH,
                target_path="results.mp4",
                callback=callback)


---

# Traning The Model

## Only on colab

In [ ]:
# Update Players_data.yaml with correct Colab paths
yaml_content = """train: /content/Football-Vision-Stats-ML/Football Advanced stats with python/football-players-detection/train
val: /content/Football-Vision-Stats-ML/Football Advanced stats with python/football-players-detection/valid
test: /content/Football-Vision-Stats-ML/Football Advanced stats with python/football-players-detection/test

nc: 4
names: ['ball', 'goalkeeper', 'player', 'referee']
"""

# Write the corrected YAML file
with open('Players_data.yaml', 'w') as f:
    f.write(yaml_content)

# Verify the fix
print("Updated YAML file:")
!cat Players_data.yaml

In [ ]:
results = PLAYER_DETECTION_MODEL.train(data="Players_data.yaml",
                                      epochs=150,
                                      imgsz=640)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls runs/detect/


---

In [ ]:
TRAINED_DETECTION_MODEL = YOLO("runs/detect/train/weights/best.pt")

